In [1]:
!pip install pysolr

  Using cached pysolr-3.9.0.tar.gz (55 kB)
  Created wheel for pysolr: filename=pysolr-3.9.0-py2.py3-none-any.whl size=19690 sha256=008929a626470bccc6742f889249f7ab544859f59f16342634b209f0cb62aec2
  Stored in directory: c:\users\heet\appdata\local\pip\cache\wheels\8f\e3\bb\f4c2a751045824a821ab25757e9762a66da88028d8f5f136ce
Successfully built pysolr


In [1]:
import os
import pysolr
import requests

CORE_NAME = "IRF22P1"
GCS_IP = "35.245.138.72"


def delete_core(core=CORE_NAME):
    print(os.system('sudo su - solr -c "/opt/solr/bin/solr delete -c {core}"'.format(core=core)))


def create_core(core=CORE_NAME):
    print(os.system(
        'sudo su - solr -c "/opt/solr/bin/solr create -c {core} -n data_driven_schema_configs"'.format(
            core=core)))


class Indexer:
    def __init__(self):
        self.solr_url = f'http://{GCS_IP}:8983/solr/'
        self.connection = pysolr.Solr(self.solr_url + CORE_NAME, always_commit=True, timeout=5000000)

    def do_initial_setup(self):
        delete_core()
        create_core()

    def create_documents(self, docs):
        print(self.connection.add(docs))

    def add_fields(self):
        data = {
            "add-field": [
                {
                    "name": "subreddit",
                    "type": "string",
                    "multiValued": False
                }, 
                {
                    "name": "full_link",
                    "type": "string",
                    "multiValued": False
                },
                {
                    "name": "title",
                    "type": "string",
                    "multiValued": False
                },
                {
                    "name": "selftext",
                    "type": "text_en",
                    "multiValued": False
                },
                 {
                    "name": "author",
                    "type": "string",
                    "multiValued": False
                },
                 {
                    "name": "is_submission",
                    "type": "boolean",
                    "multiValued": False
                },
                 {
                    "name": "topic",
                    "type": "string",
                    "multiValued": False
                },
                 {
                    "name": "created_at",
                    "type": "pdate",
                    "multiValued": False
                },
                {
                    "name": "body",
                    "type": "text_en",
                    "multiValued": False
                },
                {
                    "name": "parent_id",
                    "type": "string",
                    "multiValued": False
                },
                {
                    "name": "parent_body",
                    "type": "text_en",
                    "multiValued": False
                },
            ]
        }

        print(requests.post(self.solr_url + CORE_NAME + "/schema", json=data).json())


if __name__ == "__main__":
    i = Indexer()